In [1]:
import os
import time
import math
import numpy as np
import random
import json
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable

from utils import to_gpu, train_ngram_lm, get_ppl, create_exp_dir, Dictionary, length_sort
from models import Seq2Seq, MLP_D, MLP_D_local, MLP_G
from bleu_self import *
from bleu_test import *
import datetime
now_time = datetime.datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 

In [2]:
import argparse

parser = argparse.ArgumentParser(description='TILGAN for unconditional generation')

In [3]:
import pandas as pd
from transformers import *
from matplotlib import pyplot as plt
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

In [4]:
seed=4
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

In [5]:
#data_path="data/MS_COCO"
data_path="../../yahoo/unlabelled_small"
save="./results/yahoo_merge_assigned_results"
maxlen=16
batch_size=48
eval_batch_size = 16
noise_seq_length = 15
add_noise=True #what does this do? - question applies to most parameters
emsize=512
nhidden=512
nlayers=2
nheads=4
nff=1024
aehidden=56
noise_r=0.05
hidden_init=True
dropout=0.3
gpu=True
z_size=100
arch_g='300-300'
gan_g_activation=True
arch_d='300-300'
gan_d_local=False
gan_d_local_windowsize=3
arch_d_local='300-300'
lr_ae=0.12
lr_gan_e=1e-04
beta1=0.5
lr_gan_g=4e-04
lr_gan_d=1e-04
epochs=200
sample=True
clip=1
log_interval=100
gan_lambda=0.1
niters_gan_d=1
niters_gan_g=1
niters_gan_ae=1
niters_gan_dec=1
niters_gan_schedule=''
niters_ae=1
gan_type='kl'
enhance_dec=True
gan_gp_lambda=1
vocab_size=0
lowercase=True

In [6]:
# data="../../yahoo/yahoo_everything.csv"

label_list = ["UNK",1,2,3,4,5,6,7,8,9,10]

# df_yahoo = pd.read_csv(data)
# #df_yahoo=df_yahoo.rename(columns = {"Unnamed: 0":'label'})
# df_yahoo=df_yahoo.set_index("Unnamed: 0")
# df = df_yahoo.sample(frac=1)
# df.head()

In [7]:
# numbers=list(df['label'].unique())
# list_zeros = [0]*len(numbers)
# count_dictionary = dict(zip(numbers, list_zeros))

# values_array_train_labelled=[]
# values_array_test_labelled=[]
# values_array_test_unlabelled=[]
# values_array_train_unlabelled=[]
# values_array_unlabelled=[]
# data_all=[]
# for index, row in df.iterrows():
#     if count_dictionary[row['label']]<20:
#         count_dictionary[row['label']]=count_dictionary[row['label']]+1
#         values_array_train_labelled.append((row['question_1'],row['label']))
#     elif count_dictionary[row['label']]<60:
#         count_dictionary[row['label']]=count_dictionary[row['label']]+1
#         values_array_test_labelled.append((row['question_1'],row['label']))
#     elif count_dictionary[row['label']]<600:
#         count_dictionary[row['label']]=count_dictionary[row['label']]+1
#         values_array_unlabelled.append((row['question_1'],'UNK'))
#     elif count_dictionary[row['label']]<1600:
#         count_dictionary[row['label']]=count_dictionary[row['label']]+1
#         values_array_test_unlabelled.append((row['question_1'],'UNK'))
#     elif count_dictionary[row['label']]<7600:
#         count_dictionary[row['label']]=count_dictionary[row['label']]+1
#         values_array_train_unlabelled.append((row['question_1'],'UNK'))
#     data_all.append(row['question_1'])

In [8]:
# train_l =  values_array_train_labelled
# test_l = values_array_test_labelled
# test_u = values_array_test_unlabelled
# train_u = values_array_train_unlabelled

In [9]:
df_train_l=pd.read_csv("../../yahoo/assigned/train_l.csv", index_col="Unnamed: 0")
df_test_l=pd.read_csv("../../yahoo/assigned/test_l.csv", index_col="Unnamed: 0")
df_u=pd.read_csv("../../yahoo/assigned/u.csv", index_col="Unnamed: 0")
df_train_u=pd.read_csv("../../yahoo/assigned/train_u.csv", index_col="Unnamed: 0")#.head(30000)
df_test_u=pd.read_csv("../../yahoo/assigned/test_u.csv", index_col="Unnamed: 0")#.head(5000)
df_all = pd.concat([df_train_l, df_test_l, df_u, df_train_u, df_test_u])

In [10]:
train_l =  list(df_train_l.to_records(index=False))
test_l = list(df_test_l.to_records(index=False))
u_list = list(df_u.to_records(index=False))
test_u = list(df_test_u.to_records(index=False))
train_u = list(df_train_u.to_records(index=False))
data_all = list(df_all["0"])

In [11]:
class Corpus(object):
    def __init__(self, data_all, train_l, test_l, train_u, test_u, maxlen, vocab_size=11000, lowercase=False):
        self.dictionary = Dictionary()
        self.maxlen = maxlen
        self.lowercase = lowercase
        self.vocab_size = vocab_size
        
        self.data_all = data_all
        self.train_l = train_l
        self.test_l = test_l
        self.train_u = train_u
        self.test_u = test_u

        # make the vocabulary from training set
        self.make_vocab()
        
        self.train_l_tok = self.tokenize(self.train_l)
        self.test_l_tok = self.tokenize(self.test_l)
        self.train_u_tok = self.tokenize(self.train_u)
        self.test_u_tok = self.tokenize(self.test_u)

    def make_vocab(self):
        # Add words to the dictionary
        print(len(self.data_all))
        print(self.data_all[0])
        for line in self.data_all:
            if self.lowercase:
                # -1 to get rid of \n character
                words = line[:-1].lower().split(" ")
            else:
                words = line[:-1].split(" ")
            for word in words:
                self.dictionary.add_word(word)

        # prune the vocabulary
        self.dictionary.prune_vocab(k=self.vocab_size, cnt=True)

    def tokenize(self, data):
        """Tokenizes a text file."""
        dropped = 0
        #with open(path, 'r') as f:
        linecount = 0
        lines = []
        for line, label in data:
            linecount += 1
            if self.lowercase:
                words = line[:-1].lower().strip().split(" ")
            else:
                words = line[:-1].strip().split(" ")
            if len(words) > self.maxlen:
                dropped += 1
                continue
            words = ['<sos>'] + words
            words += ['<eos>']
            # vectorize
            vocab = self.dictionary.word2idx
            unk_idx = vocab['<oov>']
            indices = [vocab[w] if w in vocab else unk_idx for w in words]
            lines.append(indices)

        print("Number of sentences dropped: {} out of {} total".
              format(dropped, linecount))
        return lines

In [12]:
corpus = Corpus(data_all = data_all,
                train_l=train_l,
                test_l=test_l,
                train_u=train_u,
                test_u=test_u,
                maxlen=maxlen,
                vocab_size=vocab_size,
                lowercase=lowercase)

240001
how has religion affected war?
original vocab 93259; pruned to 93263
Number of sentences dropped: 0 out of 200 total
Number of sentences dropped: 1 out of 400 total
Number of sentences dropped: 39 out of 209401 total
Number of sentences dropped: 1 out of 20000 total


In [13]:
# save arguments
ntokens = len(corpus.dictionary.word2idx)
print("Vocabulary Size: {}".format(ntokens))

Vocabulary Size: 93263


In [14]:
# exp dir
create_exp_dir(os.path.join(save), ['train.py', 'models.py', 'utils.py'],
        dict=corpus.dictionary.word2idx)

Experiment dir : ./results/yahoo_merge_assigned_results


In [15]:
def logging(str, to_stdout=True):
    with open(os.path.join(save, 'log.txt'), 'a') as f:
        f.write(str + '\n')
    if to_stdout:
        print(str)

In [16]:
def batchify(data, bsz, max_len, shuffle=False, gpu=False):
    if shuffle:
        random.shuffle(data)
    nbatch = len(data) // bsz
    batches = []

    for i in range(nbatch):
        maxlen = max_len+1
        # Pad batches to maximum sequence length in batch
        batch = data[i*bsz:(i+1)*bsz]
        # subtract 1 from lengths b/c includes BOTH starts & end symbols
        lengths = [len(x)-1 for x in batch]

        # sort items by length (decreasing)
        batch, lengths = length_sort(batch, lengths)

        # source has no end symbol
        source = [x[:-1] for x in batch]
        # target has no start symbol
        target = [x[1:] for x in batch]


        for x, y in zip(source, target):
            zeros = (maxlen-len(x))*[0]
            x += zeros
            y += zeros
        source = torch.LongTensor(np.array(source))
        target = torch.LongTensor(np.array(target)).view(-1)

        if gpu:
            source = source.cuda()
            target = target.cuda()

        batches.append((source, target, lengths))

    return batches

In [17]:
test_data = batchify(corpus.test_u_tok, eval_batch_size, maxlen, shuffle=False)
train_data = batchify(corpus.train_u_tok, batch_size, maxlen,  shuffle=True)

print("Loaded data!")

Loaded data!


In [18]:
print(len(test_data))
print(len(train_data))

1249
4361


In [19]:
###############################################################################
# Build the models
###############################################################################
autoencoder = Seq2Seq(add_noise=add_noise,
                      emsize=emsize,
                      nhidden=nhidden,
                      ntokens=ntokens,
                      nlayers=nlayers,
                      nheads=nheads,
                      nff=nff,
                      aehidden=aehidden,
                      noise_r=noise_r,
                      hidden_init=hidden_init,
                      dropout=dropout,
                      gpu=True)
nlatent = aehidden * (maxlen+1)
gan_gen = MLP_G(ninput=z_size, noutput=nlatent, layers=arch_g, gan_g_activation=gan_g_activation)
gan_disc = MLP_D(ninput=nlatent, noutput=1, layers=arch_d)
gan_disc_local = MLP_D_local(ninput=gan_d_local_windowsize * aehidden, noutput=1, layers=arch_d_local)

optimizer_ae = optim.SGD(autoencoder.parameters(), lr=lr_ae)


optimizer_gan_e = optim.Adam(autoencoder.encoder.parameters(),
                             lr=lr_gan_e,
                             betas=(beta1, 0.999))
optimizer_gan_g = optim.Adam(gan_gen.parameters(),
                             lr=lr_gan_g,
                             betas=(beta1, 0.999))
optimizer_gan_d = optim.Adam(gan_disc.parameters(),
                             lr=lr_gan_d,
                             betas=(beta1, 0.999))
optimizer_gan_d_local = optim.Adam(gan_disc_local.parameters(),
                             lr=lr_gan_d,
                             betas=(beta1, 0.999))
optimizer_gan_dec = optim.Adam(autoencoder.decoder.parameters(),
                             lr=lr_gan_e,
                             betas=(beta1, 0.999))

autoencoder = autoencoder.to(device)
gan_gen = gan_gen.to(device)
gan_disc = gan_disc.to(device)
gan_disc_local = gan_disc_local.to(device)

In [20]:
def save_model():
    print("Saving models to {}".format(save))
    torch.save({
        "ae": autoencoder.state_dict(),
        "gan_g": gan_gen.state_dict(),
        "gan_d": gan_disc.state_dict(),
        "gan_d_local": gan_disc_local.state_dict()

        },
        os.path.join(save, "model.pt"))

In [21]:
def cal_norm(model):
    total_norm = 0
    for p in model.parameters():
        param_norm = p.grad.data.norm(2)
        total_norm += param_norm.item() ** 2
    total_norm = total_norm ** (1. / 2)
    return total_norm

In [22]:
def load_models():
    model_args = json.load(open(os.path.join(save, 'options.json'), 'r'))
    word2idx = json.load(open(os.path.join(save, 'vocab.json'), 'r'))
    idx2word = {v: k for k, v in word2idx.items()}

    print('Loading models from {}'.format(save))
    loaded = torch.load(os.path.join(save, "model.pt"))
    autoencoder.load_state_dict(loaded.get('ae'))
    gan_gen.load_state_dict(loaded.get('gan_g'))
    gan_disc.load_state_dict(loaded.get('gan_d'))
    gan_disc_local.load_state_dict(loaded.get('gan_d_local'))
    return model_args, idx2word, autoencoder, gan_gen, gan_disc

In [23]:
def evaluate_autoencoder(data_source, epoch):
    # Turn on evaluation mode which disables dropout.
    autoencoder.eval()
    total_loss = 0
    ntokens = len(corpus.dictionary.word2idx)
    all_accuracies = 0
    bcnt = 0
    for i, batch in enumerate(data_source):
        source, target, lengths = batch
        with torch.no_grad():
            source = Variable(source.to(device))
            target = Variable(target.to(device))
            mask = target.gt(0)
            masked_target = target.masked_select(mask)
            # examples x ntokens
            output_mask = mask.unsqueeze(1).expand(mask.size(0), ntokens)

            # output: batch x seq_len x ntokens
            output = autoencoder(source, lengths, source, add_noise=add_noise, soft=False)
            flattened_output = output.view(-1, ntokens)

            masked_output = \
                flattened_output.masked_select(output_mask).view(-1, ntokens)
            total_loss += F.cross_entropy(masked_output, masked_target)

            # accuracy
            max_vals, max_indices = torch.max(masked_output, 1)
            accuracy = torch.mean(max_indices.eq(masked_target).float()).data.item()
            all_accuracies += accuracy
            bcnt += 1

        aeoutf = os.path.join(save, "autoencoder.txt")
        with open(aeoutf, "w") as f:
            max_values, max_indices = torch.max(output, 2)
            max_indices = \
                max_indices.view(output.size(0), -1).data.cpu().numpy()
            target = target.view(output.size(0), -1).data.cpu().numpy()
            for t, idx in zip(target, max_indices):
                # real sentence
                chars = " ".join([corpus.dictionary.idx2word[x] for x in t])
                f.write(chars + '\n')
                # autoencoder output sentence
                chars = " ".join([corpus.dictionary.idx2word[x] for x in idx])
                f.write(chars + '\n'*2)

    return total_loss.item() / len(data_source), all_accuracies/bcnt

In [24]:
def gen_fixed_noise(noise, to_save):
    gan_gen.eval()
    autoencoder.eval()

    fake_hidden = gan_gen(noise)
    max_indices = autoencoder.generate(fake_hidden, maxlen, sample=sample)

    with open(to_save, "w") as f:
        max_indices = max_indices.data.cpu().numpy()
        for idx in max_indices:
            # generated sentence
            words = [corpus.dictionary.idx2word[x] for x in idx]
            # truncate sentences to first occurrence of <eos>
            truncated_sent = []
            for w in words:
                if w != '<eos>':
                    truncated_sent.append(w)
                else:
                    break
            chars = " ".join(truncated_sent)
            f.write(chars + '\n')

In [25]:
def gen_fixed_noise_new(noise):
    gan_gen.eval()
    autoencoder.eval()

    fake_hidden = gan_gen(noise)
    max_indices = autoencoder.generate(fake_hidden, maxlen, sample=sample)
    
    sent_list = []
    
    #with open(to_save, "w") as f:
    max_indices = max_indices.data.cpu().numpy()
    for idx in max_indices:
        # generated sentence
        words = [corpus.dictionary.idx2word[x] for x in idx]
        # truncate sentences to first occurrence of <eos>
        truncated_sent = []
        for w in words:
            if w != '<eos>':
                truncated_sent.append(w)
            else:
                break
        chars = " ".join(truncated_sent)
        #f.write(chars + '\n')
        sent_list.append(chars)
    #print(sent_list)
    return sent_list

In [26]:
def eval_bleu(gen_text_savepath):
    selfbleu = bleu_self(gen_text_savepath)
    real_text = os.path.join(data_path, "test.txt")
    testbleu = bleu_test(real_text, gen_text_savepath)
    return selfbleu, testbleu

In [27]:
def train_ae(epoch, batch, total_loss_ae, start_time, i):
    '''Train AE with the negative log-likelihood loss'''
    autoencoder.train()
    optimizer_ae.zero_grad()

    source, target, lengths = batch
    source = Variable(source.to(device))
    target = Variable(target.to(device))
    output = autoencoder(source, lengths, source, add_noise=add_noise, soft=False)

    mask = target.gt(0)
    masked_target = target.masked_select(mask)
    output_mask = mask.unsqueeze(1).expand(mask.size(0), ntokens)
    flat_output = output.view(-1, ntokens)
    masked_output = flat_output.masked_select(output_mask).view(-1, ntokens)
    loss = F.cross_entropy(masked_output, masked_target)
    loss.backward()
    torch.nn.utils.clip_grad_norm(autoencoder.parameters(), clip)
    train_ae_norm = cal_norm(autoencoder)
    optimizer_ae.step()

    total_loss_ae += loss.data.item()
    if i % log_interval == 0:
        probs = F.softmax(masked_output, dim=-1)
        max_vals, max_indices = torch.max(probs, 1)
        accuracy = torch.mean(max_indices.eq(masked_target).float()).data.item()
        cur_loss = total_loss_ae / log_interval
        elapsed = time.time() - start_time
        logging('| epoch {:3d} | {:5d}/{:5d} batches | lr {:08.6f} | ms/batch {:5.2f} | '
                'loss {:5.2f} | ppl {:8.2f} | acc {:8.2f} | train_ae_norm {:8.2f}'.format(
                epoch, i, len(train_data), 0,
                elapsed * 1000 / log_interval,
                cur_loss, math.exp(cur_loss), accuracy, train_ae_norm))

        total_loss_ae = 0
        start_time = time.time()
    return total_loss_ae, start_time

In [28]:
def train_gan_g(gan_type='kl'):
    gan_gen.train()
    optimizer_gan_g.zero_grad()

    z = Variable(torch.Tensor(batch_size, z_size).normal_(0, 1).to(device))
    fake_hidden = gan_gen(z)
    fake_score = gan_disc(fake_hidden)

    if gan_d_local:
        idx = random.randint(0, maxlen - gan_d_local_windowsize)
        fake_hidden_local = fake_hidden[:, idx * aehidden : (idx + gan_d_local_windowsize) * aehidden]
        fake_score_local = gan_disc_local(fake_hidden_local)

        if gan_type == 'kl':
            errG = -(torch.exp(fake_score.detach()).clamp(0.5, 2) * fake_score).mean() -(torch.exp(fake_score_local.detach()).clamp(0.5, 2) * fake_score_local).mean()
        else:
            errG = -fake_score.mean() -fake_score_local.mean()
    else:
        if gan_type == 'kl':
            errG = -(torch.exp(fake_score.detach()).clamp(0.5, 2) * fake_score).mean()
        else:
            errG = -fake_score.mean()


    errG *= gan_lambda
    errG.backward()
    optimizer_gan_g.step()

    return errG

In [29]:
def train_gan_dec(gan_type='kl'):
    autoencoder.decoder.train()
    optimizer_gan_dec.zero_grad()

    z = Variable(torch.Tensor(batch_size, z_size).normal_(0, 1).to(device))
    fake_hidden = gan_gen(z)

    # 1. decoder  - soft distribution
    enhance_source, max_indices= autoencoder.generate_enh_dec(fake_hidden, maxlen, sample=sample)
    # 2. soft distribution - > encoder  -> fake_hidden
    enhance_hidden = autoencoder(enhance_source, None, max_indices, add_noise=add_noise, soft=True, encode_only=True)
    fake_score = gan_disc(enhance_hidden)

    if gan_d_local:
        idx = random.randint(0, maxlen - gan_d_local_windowsize)
        fake_hidden_local = fake_hidden[:, idx * aehidden : (idx + gan_d_local_windowsize) * aehidden]
        fake_score_local = gan_disc_local(fake_hidden_local)

        if gan_type == 'kl':
            errG = -(torch.exp(fake_score.detach()).clamp(0.5, 2) * fake_score).mean() -(torch.exp(fake_score_local.detach()).clamp(0.5, 2) * fake_score_local).mean()
        else:
            errG = -fake_score.mean() -fake_score_local.mean()
    else:
        if gan_type == 'kl':
            errG = -(torch.exp(fake_score.detach()).clamp(0.5, 2) * fake_score).mean()
        else:
            errG = -fake_score.mean()


    errG *= gan_lambda
    errG.backward()
    optimizer_gan_dec.step()

    return errG

In [30]:
def grad_hook(grad):
    return grad * gan_lambda

In [31]:
''' Steal from https://github.com/caogang/wgan-gp/blob/master/gan_cifar10.py '''
def calc_gradient_penalty(netD, real_data, fake_data):
    bsz = real_data.size(0)
    alpha = torch.rand(bsz, 1)
    alpha = alpha.expand(bsz, real_data.size(1))  # only works for 2D XXX
    alpha = alpha.to(device)
    interpolates = alpha * real_data + ((1 - alpha) * fake_data)
    interpolates = Variable(interpolates, requires_grad=True)
    disc_interpolates = netD(interpolates)

    gradients = torch.autograd.grad(outputs=disc_interpolates, inputs=interpolates,
                                    grad_outputs=torch.ones(disc_interpolates.size()).to(device),
                                    create_graph=True, retain_graph=True, only_inputs=True)[0]
    gradients = gradients.view(gradients.size(0), -1)

    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean() * gan_gp_lambda
    return gradient_penalty

In [32]:
def train_gan_d(batch, gan_type='kl'):
    gan_disc.train()
    gan_disc_local.train()
    optimizer_gan_d.zero_grad()
    optimizer_gan_d_local.zero_grad()

    # + samples
    source, target, lengths = batch
    source = Variable(source.to(device))
    target = Variable(target.to(device))
    real_hidden = autoencoder(source, lengths, source, add_noise=add_noise, soft=False, encode_only=True)
    real_score = gan_disc(real_hidden.detach())

    idx = random.randint(0, maxlen - gan_d_local_windowsize)
    if gan_d_local:
        real_hidden_local = real_hidden[:, idx * aehidden : (idx + gan_d_local_windowsize) * aehidden]
        real_score_local = gan_disc_local(real_hidden_local)
        real_score += real_score_local


    if gan_type == 'wgan':
        errD_real = -real_score.mean()
    else: # kl or all
        errD_real = F.softplus(-real_score).mean()
    errD_real.backward()

    # - samples
    z = Variable(torch.Tensor(batch_size, z_size).normal_(0, 1).to(device))
    fake_hidden = gan_gen(z)
    fake_score = gan_disc(fake_hidden.detach())

    if gan_d_local:
        fake_hidden_local = fake_hidden[:, idx * aehidden : (idx + gan_d_local_windowsize) * aehidden]
        fake_score_local = gan_disc_local(fake_hidden_local)
        fake_score += fake_score_local

    if gan_type == 'wgan':
        errD_fake = fake_score.mean()
    else:  # kl or all
        errD_fake = F.softplus(fake_score).mean()
    errD_fake.backward()

    # gradient penalty
    if gan_type == 'wgan':
        gradient_penalty = calc_gradient_penalty(gan_disc, real_hidden.data, fake_hidden.data)
        gradient_penalty.backward()

    optimizer_gan_d.step()
    optimizer_gan_d_local.step()
    return errD_real + errD_fake, errD_real, errD_fake

In [33]:
def train_gan_d_into_ae(batch):
    autoencoder.train()
    optimizer_gan_e.zero_grad()

    source, target, lengths = batch
    source = Variable(source.to(device))
    target = Variable(target.to(device))
    real_hidden = autoencoder(source, lengths, source, add_noise=add_noise, soft=False, encode_only=True)

    if gan_d_local:
        idx = random.randint(0, maxlen - gan_d_local_windowsize)
        real_hidden_local = real_hidden[:, idx * aehidden : (idx + gan_d_local_windowsize) * aehidden]
        real_score_local = gan_disc_local(real_hidden_local)
        errD_real = gan_disc(real_hidden).mean() + real_score_local.mean()
    else:
        errD_real = gan_disc(real_hidden).mean()

    errD_real *= gan_lambda
    errD_real.backward()
    torch.nn.utils.clip_grad_norm(autoencoder.parameters(), clip)

    optimizer_gan_e.step()
    return errD_real

In [34]:
#--------------------------------
#  Transformer parameters
#--------------------------------
max_seq_length = 20
batch_size_d = 48

#--------------------------------
#  GAN-BERT specific parameters
#--------------------------------
# number of hidden layers in the generator, 
# each of the size of the output space
#num_hidden_layers_g = 1; 
# number of hidden layers in the discriminator, 
# each of the size of the input space
num_hidden_layers_d = 1; 
# size of the generator's input noisy vectors
noise_size = 100
# dropout to be applied to discriminator's input vectors
out_dropout_rate = 0.2

# Replicate labeled data to balance poorly represented datasets, 
# e.g., less than 1% of labeled material
apply_balance = True

#--------------------------------
#  Optimization parameters
#--------------------------------
learning_rate_discriminator = 5e-6 #5e-6?
#learning_rate_generator = 5e-5
epsilon = 1e-8
num_train_epochs = 50
multi_gpu = True
# Scheduler
apply_scheduler = False
warmup_proportion = 0.1
# Print
print_each_n_step = 10

#--------------------------------
#  Adopted Tranformer model
#--------------------------------
# Since this version is compatible with Huggingface transformers, you can uncomment
# (or add) transformer models compatible with GAN

model_name = "bert-base-cased"
#model_name = "bert-base-uncased"
#model_name = "roberta-base"
#model_name = "albert-base-v2"
#model_name = "xlm-roberta-base"
#model_name = "amazon/bort"
#model_name="google/electra-large-discriminator"
#model_name="google/electra-small-discriminator"
#model_name="microsoft/deberta-v2-xxlarge"
#model_name="microsoft/deberta-v3-base"
#model_name = "google/electra-base-discriminator"

In [35]:
transformer = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

loading configuration file https://huggingface.co/bert-base-cased/resolve/main/config.json from cache at /home/harry/.cache/huggingface/transformers/a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.a64a22196690e0e82ead56f388a3ef3a50de93335926ccfa20610217db589307
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.18.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading weights file https://huggingface.co/bert-base-cased/re

In [36]:
def generate_data_loader(input_examples, label_masks, label_map, do_shuffle = False, balance_label_examples = False):
  '''
  Generate a Dataloader given the input examples, eventually masked if they are 
  to be considered NOT labeled.
  '''
  examples = []

  # Count the percentage of labeled examples  
  num_labeled_examples = 0
  for label_mask in label_masks:
    if label_mask: 
      num_labeled_examples += 1
  label_mask_rate = num_labeled_examples/len(input_examples)

  # if required it applies the balance
  for index, ex in enumerate(input_examples): 
    if label_mask_rate == 1 or not balance_label_examples:
      examples.append((ex, label_masks[index]))
    else:
      # IT SIMULATE A LABELED EXAMPLE
      if label_masks[index]:
        balance = int(1/label_mask_rate)
        balance = int(math.log(balance,2))
        if balance < 1:
          balance = 1
        for b in range(0, int(balance)):
          examples.append((ex, label_masks[index]))
      else:
        examples.append((ex, label_masks[index]))
  
  #-----------------------------------------------
  # Generate input examples to the Transformer
  #-----------------------------------------------
  input_ids = []
  input_mask_array = []
  label_mask_array = []
  label_id_array = []

  # Tokenization 
  for (text, label_mask) in examples:
    encoded_sent = tokenizer.encode(text[0], add_special_tokens=True, max_length=max_seq_length, padding="max_length", truncation=True)
    input_ids.append(encoded_sent)
    label_id_array.append(label_map[text[1]])
    label_mask_array.append(label_mask)
  
  # Attention to token (to ignore padded input wordpieces)
  for sent in input_ids:
    att_mask = [int(token_id > 0) for token_id in sent]                          
    input_mask_array.append(att_mask)
  # Convertion to Tensor
  input_ids = torch.tensor(input_ids) 
  input_mask_array = torch.tensor(input_mask_array)
  label_id_array = torch.tensor(label_id_array, dtype=torch.long)
  label_mask_array = torch.tensor(label_mask_array)

  # Building the TensorDataset
  dataset = TensorDataset(input_ids, input_mask_array, label_id_array, label_mask_array)

  if do_shuffle:
    sampler = RandomSampler
  else:
    sampler = SequentialSampler

  # Building the DataLoader
  return DataLoader(
              dataset,  # The training samples.
              sampler = sampler(dataset), 
              batch_size = batch_size) # Trains with this batch size.

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [37]:
def generate_data_fake(input_examples):
  '''
  Generate a Dataloader given the input examples, eventually masked if they are 
  to be considered NOT labeled.
  '''
  
  #-----------------------------------------------
  # Generate input examples to the Transformer
  #-----------------------------------------------
  input_ids = []
  input_mask_array = []

  # Tokenization 
  for text in input_examples:
    encoded_sent = tokenizer.encode(text, add_special_tokens=True, max_length=max_seq_length, padding="max_length", truncation=True)
    input_ids.append(encoded_sent)
  
  # Attention to token (to ignore padded input wordpieces)
  for sent in input_ids:
    att_mask = [int(token_id > 0) for token_id in sent]                          
    input_mask_array.append(att_mask)
  # Convertion to Tensor
  input_ids = torch.tensor(input_ids) 
  input_mask_array = torch.tensor(input_mask_array)

  # Building the DataLoader
  return input_ids, input_mask_array # Trains with this batch size.

In [38]:
#Load the examples
labeled_examples = train_l
unlabeled_examples = u_list
test_examples = test_l

In [39]:
label_map = {}
for (i, label) in enumerate(label_list):
  label_map[label] = i
#------------------------------
#   Load the train dataset
#------------------------------
train_examples = labeled_examples
#The labeled (train) dataset is assigned with a mask set to True
train_label_masks = np.ones(len(labeled_examples), dtype=bool)
#If unlabel examples are available
if unlabeled_examples:
  train_examples = train_examples + unlabeled_examples
  #The unlabeled (train) dataset is assigned with a mask set to False
  tmp_masks = np.zeros(len(unlabeled_examples), dtype=bool)
  train_label_masks = np.concatenate([train_label_masks,tmp_masks])

train_dataloader = generate_data_loader(train_examples, train_label_masks, label_map, do_shuffle = True, balance_label_examples = apply_balance)

#------------------------------
#   Load the test dataset
#------------------------------
#The labeled (test) dataset is assigned with a mask set to True
test_label_masks = np.ones(len(test_examples), dtype=bool)

test_dataloader = generate_data_loader(test_examples, test_label_masks, label_map, do_shuffle = False, balance_label_examples = False)

In [40]:
#------------------------------
#   The Discriminator
#   https://www.aclweb.org/anthology/2020.acl-main.191/
#   https://github.com/crux82/ganbert
#------------------------------
class Discriminator(nn.Module):
    def __init__(self, input_size=512, hidden_sizes=[512], num_labels=2, dropout_rate=0.1):
        super(Discriminator, self).__init__()
        self.input_dropout = nn.Dropout(p=dropout_rate)
        layers = []
        hidden_sizes = [input_size] + hidden_sizes
        for i in range(len(hidden_sizes)-1):
            layers.extend([nn.Linear(hidden_sizes[i], hidden_sizes[i+1]), nn.LeakyReLU(0.2, inplace=True), nn.Dropout(dropout_rate)])

        self.layers = nn.Sequential(*layers) #per il flatten
        self.logit = nn.Linear(hidden_sizes[-1],num_labels+1) # +1 for the probability of this sample being fake/real.
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, input_rep):
        input_rep = self.input_dropout(input_rep)
        last_rep = self.layers(input_rep)
        logits = self.logit(last_rep)
        probs = self.softmax(logits)
        return last_rep, logits, probs

In [41]:
# The config file is required to get the dimension of the vector produced by 
# the underlying transformer
config = AutoConfig.from_pretrained(model_name)
hidden_size = int(config.hidden_size)
# Define the number and width of hidden layers
#hidden_levels_g = [hidden_size for i in range(0, num_hidden_layers_g)]
hidden_levels_d = [hidden_size for i in range(0, num_hidden_layers_d)]

#-------------------------------------------------
#   Instantiate the Generator and Discriminator
#-------------------------------------------------
#generator = Generator(noise_size=noise_size, output_size=hidden_size, hidden_sizes=hidden_levels_g, dropout_rate=out_dropout_rate)
discriminator = Discriminator(input_size=hidden_size, hidden_sizes=hidden_levels_d, num_labels=len(label_list), dropout_rate=out_dropout_rate)

# Put everything in the GPU if available
if torch.cuda.is_available():    
  #generator.cuda()
  discriminator.cuda()
  transformer.cuda()
  if multi_gpu:
    transformer = torch.nn.DataParallel(transformer)

# print(config)

loading configuration file https://huggingface.co/bert-base-cased/resolve/main/config.json from cache at /home/harry/.cache/huggingface/transformers/a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.a64a22196690e0e82ead56f388a3ef3a50de93335926ccfa20610217db589307
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.18.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}



In [42]:
label_map = {}
for (i, label) in enumerate(label_list):
  label_map[label] = i
#------------------------------
#   Load the train dataset
#------------------------------
train_examples = labeled_examples
#The labeled (train) dataset is assigned with a mask set to True
train_label_masks = np.ones(len(labeled_examples), dtype=bool)
#If unlabel examples are available
if unlabeled_examples:
  train_examples = train_examples + unlabeled_examples
  #The unlabeled (train) dataset is assigned with a mask set to False
  tmp_masks = np.zeros(len(unlabeled_examples), dtype=bool)
  train_label_masks = np.concatenate([train_label_masks,tmp_masks])

train_dataloader = generate_data_loader(train_examples, train_label_masks, label_map, do_shuffle = True, balance_label_examples = apply_balance)

#------------------------------
#   Load the test dataset
#------------------------------
#The labeled (test) dataset is assigned with a mask set to True
test_label_masks = np.ones(len(test_examples), dtype=bool)

test_dataloader = generate_data_loader(test_examples, test_label_masks, label_map, do_shuffle = False, balance_label_examples = False)

In [43]:
training_stats = []

accuracy_array=[]

# Measure the total training time for the whole run.
total_t0 = time.time()

#models parameters
transformer_vars = [i for i in transformer.parameters()]
d_vars = transformer_vars + [v for v in discriminator.parameters()]
#g_vars = [v for v in generator.parameters()]

#optimizer
dis_optimizer = torch.optim.AdamW(d_vars, lr=learning_rate_discriminator)
#gen_optimizer = torch.optim.AdamW(g_vars, lr=learning_rate_generator) 

#scheduler
if apply_scheduler:
  num_train_examples = len(train_examples)
  num_train_steps = int(num_train_examples / batch_size * num_train_epochs)
  num_warmup_steps = int(num_train_steps * warmup_proportion)

  scheduler_d = get_constant_schedule_with_warmup(dis_optimizer, 
                                           num_warmup_steps = num_warmup_steps)
  scheduler_g = get_constant_schedule_with_warmup(gen_optimizer, 
                                           num_warmup_steps = num_warmup_steps)

In [44]:
def train():
    logging("Training")
    train_data = batchify(corpus.train_u_tok, batch_size, maxlen, shuffle=True)

    # gan: preparation
    if niters_gan_schedule != "":
        gan_schedule = [int(x) for x in niters_gan_schedule.split("-")]
    else:
        gan_schedule = []
    niter_gan = 1
    fixed_noise = Variable(torch.ones(eval_batch_size, z_size).normal_(0, 1).to(device))

    for epoch in range(1, epochs+1):
        # update gan training schedule
        if epoch in gan_schedule:
            niter_gan += 1
            logging("GAN training loop schedule: {}".format(niter_gan))

        total_loss_ae = 0
        epoch_start_time = time.time()
        start_time = time.time()
        niter = 0
        niter_g = 1
        print("Train classification discriminator")
        # ========================================
        #               Training
        # ========================================
        # Perform one full pass over the training set.
        print("")
        print('======== Epoch {:} / {:} ========'.format(epoch, epochs))
        print('Training...')

        # Measure how long the training epoch takes.
        t0 = time.time()

        # Reset the total loss for this epoch.
        tr_g_loss = 0
        tr_d_loss = 0

        # Put the model into training mode.
        transformer.train() 
        #generator.train()
        discriminator.train()

        # For each batch of training data...
        for step, batch in enumerate(train_dataloader):

            # Progress update every print_each_n_step batches.
            if step % print_each_n_step == 0 and not step == 0:
                # Calculate elapsed time in minutes.
                elapsed = format_time(time.time() - t0)

                # Report progress.
                print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

            # Unpack this training batch from our dataloader. 
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)
            b_label_mask = batch[3].to(device)

            real_batch_size = b_input_ids.shape[0]

            # Encode real data in the Transformer
            model_outputs = transformer(b_input_ids, attention_mask=b_input_mask)
            hidden_states = model_outputs.last_hidden_state[:,0,:] 
            #hidden_states = model_outputs[-1]
            #print("  Number of real sentences (labelled and unlabelled): {}".format(len(hidden_states)))
            
            # Generate fake data that should have the same distribution of the ones
            # encoded by the transformer. 
            # First noisy input are used in input to the Generator
            fixed_noise = Variable(torch.ones(real_batch_size, 100).normal_(0, 1).to(device))
            fake_sentences = gen_fixed_noise_new(fixed_noise)
            #print("  Number of generated sentences: {}".format(len(fake_sentences)))

            b_input_ids_fake, b_input_mask_fake = generate_data_fake(fake_sentences)
            model_outputs_fake = transformer(b_input_ids_fake, attention_mask=b_input_mask_fake)
            hidden_states_fake = model_outputs_fake.last_hidden_state[:,0,:] 
            #hidden_states_fake = model_outputs_fake[-1]

            #noise = torch.zeros(real_batch_size, noise_size, device=device).uniform_(0, 1)
            # Gnerate Fake data
            #gen_rep = generator(noise)
            #print("Length of generator output {}".format(len(gen_rep)))
            #print("Length of single generator output {}".format(len(gen_rep[0])))

            # Generate the output of the Discriminator for real and fake data.
            # First, we put together the output of the tranformer and the generator
            disciminator_input = torch.cat([hidden_states, hidden_states_fake], dim=0)
            # Then, we select the output of the disciminator
            features, logits, probs = discriminator(disciminator_input)

            # Finally, we separate the discriminator's output for the real and fake
            # data
            features_list = torch.split(features, real_batch_size)
            D_real_features = features_list[0]
            D_fake_features = features_list[1]

            logits_list = torch.split(logits, real_batch_size)
            D_real_logits = logits_list[0]
            D_fake_logits = logits_list[1]

            probs_list = torch.split(probs, real_batch_size)
            D_real_probs = probs_list[0]
            D_fake_probs = probs_list[1]

            #---------------------------------
            #  LOSS evaluation
            #---------------------------------
            # Generator's LOSS estimation
            g_loss_d = -1 * torch.mean(torch.log(1 - D_fake_probs[:,-1] + epsilon))
            g_feat_reg = torch.mean(torch.pow(torch.mean(D_real_features, dim=0) - torch.mean(D_fake_features, dim=0), 2))
            g_loss = g_loss_d + g_feat_reg

            # Disciminator's LOSS estimation
            logits = D_real_logits[:,0:-1]
            log_probs = F.log_softmax(logits, dim=-1)
            # The discriminator provides an output for labeled and unlabeled real data
            # so the loss evaluated for unlabeled data is ignored (masked)
            label2one_hot = torch.nn.functional.one_hot(b_labels, len(label_list))
            per_example_loss = -torch.sum(label2one_hot * log_probs, dim=-1)
            per_example_loss = torch.masked_select(per_example_loss, b_label_mask.to(device))
            labeled_example_count = per_example_loss.type(torch.float32).numel()

            # It may be the case that a batch does not contain labeled examples, 
            # so the "supervised loss" in this case is not evaluated
            if labeled_example_count == 0:
              D_L_Supervised = 0
            else:
              D_L_Supervised = torch.div(torch.sum(per_example_loss.to(device)), labeled_example_count)

            D_L_unsupervised1U = -1 * torch.mean(torch.log(1 - D_real_probs[:, -1] + epsilon))
            D_L_unsupervised2U = -1 * torch.mean(torch.log(D_fake_probs[:, -1] + epsilon))
            d_loss = D_L_Supervised + D_L_unsupervised1U + D_L_unsupervised2U

            #---------------------------------
            #  OPTIMIZATION
            #---------------------------------
            # Avoid gradient accumulation
            #gen_optimizer.zero_grad()
            dis_optimizer.zero_grad()

            # Calculate weigth updates
            # retain_graph=True is required since the underlying graph will be deleted after backward
            g_loss.backward(retain_graph=True)
            d_loss.backward() 

            # Apply modifications
            #gen_optimizer.step()
            dis_optimizer.step()

            # A detail log of the individual losses
            #print("{0:.4f}\t{1:.4f}\t{2:.4f}\t{3:.4f}\t{4:.4f}".
            #      format(D_L_Supervised, D_L_unsupervised1U, D_L_unsupervised2U,
            #             g_loss_d, g_feat_reg))

            # Save the losses to print them later
            tr_g_loss += g_loss.item()
            tr_d_loss += d_loss.item()

            # Update the learning rate with the scheduler
            if apply_scheduler:
              scheduler_d.step()
              #scheduler_g.step()

        # Calculate the average loss over all of the batches.
        avg_train_loss_g = tr_g_loss / len(train_dataloader)
        avg_train_loss_d = tr_d_loss / len(train_dataloader)             

        # Measure how long this epoch took.
        training_time = format_time(time.time() - t0)

        print("")
        print("  Average training loss generetor: {0:.3f}".format(avg_train_loss_g))
        print("  Average training loss discriminator: {0:.3f}".format(avg_train_loss_d))
        print("  Training epcoh took: {:}".format(training_time))

        # ========================================
        #     TEST ON THE EVALUATION DATASET
        # ========================================
        # After the completion of each training epoch, measure our performance on
        # our test set.
        print("")
        print("Running Test...")

        t0 = time.time()

        # Put the model in evaluation mode--the dropout layers behave differently
        # during evaluation.
        transformer.eval() #maybe redundant
        discriminator.eval()
        #generator.eval()

        # Tracking variables 
        total_test_accuracy = 0

        total_test_loss = 0
        nb_test_steps = 0

        all_preds = []
        all_labels_ids = []

        #loss
        nll_loss = torch.nn.CrossEntropyLoss(ignore_index=-1)

        # Evaluate data for one epoch
        for batch in test_dataloader:

            # Unpack this training batch from our dataloader. 
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)

            # Tell pytorch not to bother with constructing the compute graph during
            # the forward pass, since this is only needed for backprop (training).
            with torch.no_grad():        
                model_outputs = transformer(b_input_ids, attention_mask=b_input_mask)
                hidden_states = model_outputs.last_hidden_state[:,0,:] 
                #hidden_states = model_outputs[-1]
                _, logits, probs = discriminator(hidden_states)
                ###log_probs = F.log_softmax(probs[:,1:], dim=-1)
                filtered_logits = logits[:,0:-1]
                # Accumulate the test loss.
                total_test_loss += nll_loss(filtered_logits, b_labels)

            # Accumulate the predictions and the input labels
            _, preds = torch.max(filtered_logits, 1)
            all_preds += preds.detach().cpu()
            all_labels_ids += b_labels.detach().cpu()

        # Report the final accuracy for this validation run.
        all_preds = torch.stack(all_preds).numpy()
        all_labels_ids = torch.stack(all_labels_ids).numpy()
        test_accuracy = np.sum(all_preds == all_labels_ids) / len(all_preds)
        print("  Accuracy: {0:.3f}".format(test_accuracy))

        # Calculate the average loss over all of the batches.
        avg_test_loss = total_test_loss / len(test_dataloader)
        avg_test_loss = avg_test_loss.item()

        # Measure how long the validation run took.
        test_time = format_time(time.time() - t0)

        print("  Test Loss: {0:.3f}".format(avg_test_loss))
        print("  Test took: {:}".format(test_time))

        # Record all statistics from this epoch.
        training_stats.append(
            {
                'epoch': epoch + 1,
                'Training Loss generator': avg_train_loss_g,
                'Training Loss discriminator': avg_train_loss_d,
                'Valid. Loss': avg_test_loss,
                'Valid. Accur.': test_accuracy,
                'Training Time': training_time,
                'Test Time': test_time
            }
        )

        accuracy_array.append(test_accuracy)
        print("Train other shit")
        while niter < len(train_data):
            # train ae
            for i in range(niters_ae):
                if niter >= len(train_data):
                    break  # end of epoch
                total_loss_ae, start_time = train_ae(epoch, train_data[niter],
                                total_loss_ae, start_time, niter)
                niter += 1
            # train gan
            for k in range(niter_gan):
                for i in range(niters_gan_d):
                    errD, errD_real, errD_fake = train_gan_d(
                            train_data[random.randint(0, len(train_data)-1)], gan_type)
                for i in range(niters_gan_ae):
                    train_gan_d_into_ae(train_data[random.randint(0, len(train_data)-1)])
                for i in range(niters_gan_g):
                    errG = train_gan_g(gan_type)
                if enhance_dec:
                    for i in range(niters_gan_dec):
                        errG_enh_dec = train_gan_dec()
                else:
                    errG_enh_dec = torch.Tensor([0])

            niter_g += 1
            if niter_g % log_interval == 0:
                logging('[{}/{}][{}/{}] Loss_D: {:.8f} (Loss_D_real: {:.8f} '
                        'Loss_D_fake: {:.8f}) Loss_G: {:.8f} Loss_Enh_Dec: {:.8f}'.format(
                         epoch, epochs, niter, len(train_data),
                         errD.data.item(), errD_real.data.item(),
                         errD_fake.data.item(), errG.data.item(), errG_enh_dec.data.item()))
        # eval
        test_loss, accuracy = evaluate_autoencoder(test_data, epoch)
        logging('| end of epoch {:3d} | time: {:5.2f}s | test loss {:5.2f} | '
                'test ppl {:5.2f} | acc {:3.3f}'.format(epoch,
                (time.time() - epoch_start_time), test_loss,
                math.exp(test_loss), accuracy))

        gen_text_savepath = os.path.join(save, "{:03d}_examplar_gen".format(epoch))
        gen_fixed_noise(fixed_noise, gen_text_savepath)
        if epoch % 5 == 0 or epoch % 4 == 0 or (epochs - epoch) <=2:
            selfbleu, testbleu = eval_bleu(gen_text_savepath)
            logging('bleu_self: [{:.8f},{:.8f},{:.8f},{:.8f},{:.8f}]'.format(selfbleu[0], selfbleu[1], selfbleu[2], selfbleu[3], selfbleu[4]))
            logging('bleu_test: [{:.8f},{:.8f},{:.8f},{:.8f},{:.8f}]'.format(testbleu[0], testbleu[1], testbleu[2], testbleu[3], testbleu[4]))

        if epoch % 15 == 0 or epoch == epochs-1:
            logging("New saving model: epoch {:03d}.".format(epoch))
            save_model()

In [ ]:
train()

Training
Train classification discriminator

======== Epoch 1 / 200 ========
Training...
  Batch    10  of    230.    Elapsed: 0:00:04.
  Batch    20  of    230.    Elapsed: 0:00:08.
  Batch    30  of    230.    Elapsed: 0:00:11.
  Batch    40  of    230.    Elapsed: 0:00:15.
  Batch    50  of    230.    Elapsed: 0:00:19.
  Batch    60  of    230.    Elapsed: 0:00:23.
  Batch    70  of    230.    Elapsed: 0:00:27.
  Batch    80  of    230.    Elapsed: 0:00:30.
  Batch    90  of    230.    Elapsed: 0:00:34.
  Batch   100  of    230.    Elapsed: 0:00:38.
  Batch   110  of    230.    Elapsed: 0:00:42.
  Batch   120  of    230.    Elapsed: 0:00:46.
  Batch   130  of    230.    Elapsed: 0:00:49.
  Batch   140  of    230.    Elapsed: 0:00:53.
  Batch   150  of    230.    Elapsed: 0:00:57.
  Batch   160  of    230.    Elapsed: 0:01:01.
  Batch   170  of    230.    Elapsed: 0:01:05.
  Batch   180  of    230.    Elapsed: 0:01:09.
  Batch   190  of    230.    Elapsed: 0:01:12.
  Batch   200  of 

/home/harry/venv/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
/home/harry/venv/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.


[1/200][99/4361] Loss_D: 1.38524914 (Loss_D_real: 0.69120073 Loss_D_fake: 0.69404840) Loss_G: -0.00016868 Loss_Enh_Dec: -0.00040080
| epoch   1 |   100/ 4361 batches | lr 0.000000 | ms/batch 399.08 | loss  8.57 | ppl  5260.61 | acc     0.19 | train_ae_norm     1.00
[1/200][199/4361] Loss_D: 1.38535583 (Loss_D_real: 0.69144821 Loss_D_fake: 0.69390762) Loss_G: -0.00013183 Loss_Enh_Dec: -0.00032994
| epoch   1 |   200/ 4361 batches | lr 0.000000 | ms/batch 398.60 | loss  7.58 | ppl  1949.98 | acc     0.23 | train_ae_norm     1.00
[1/200][299/4361] Loss_D: 1.38564086 (Loss_D_real: 0.69203019 Loss_D_fake: 0.69361067) Loss_G: -0.00004620 Loss_Enh_Dec: -0.00020663
| epoch   1 |   300/ 4361 batches | lr 0.000000 | ms/batch 398.52 | loss  7.21 | ppl  1350.46 | acc     0.22 | train_ae_norm     1.00
[1/200][399/4361] Loss_D: 1.38577974 (Loss_D_real: 0.69268084 Loss_D_fake: 0.69309890) Loss_G: -0.00000916 Loss_Enh_Dec: -0.00005714
| epoch   1 |   400/ 4361 batches | lr 0.000000 | ms/batch 397.57 |

[1/200][3199/4361] Loss_D: 1.38636816 (Loss_D_real: 0.69314611 Loss_D_fake: 0.69322205) Loss_G: 0.00000650 Loss_Enh_Dec: -0.00000008
| epoch   1 |  3200/ 4361 batches | lr 0.000000 | ms/batch 396.04 | loss  5.15 | ppl   172.96 | acc     0.42 | train_ae_norm     1.00
[1/200][3299/4361] Loss_D: 1.38643909 (Loss_D_real: 0.69333905 Loss_D_fake: 0.69310009) Loss_G: 0.00003178 Loss_Enh_Dec: 0.00002790
| epoch   1 |  3300/ 4361 batches | lr 0.000000 | ms/batch 396.07 | loss  5.13 | ppl   169.13 | acc     0.44 | train_ae_norm     1.00
[1/200][3399/4361] Loss_D: 1.38636720 (Loss_D_real: 0.69319361 Loss_D_fake: 0.69317359) Loss_G: 0.00001398 Loss_Enh_Dec: 0.00000927
| epoch   1 |  3400/ 4361 batches | lr 0.000000 | ms/batch 396.37 | loss  5.08 | ppl   161.00 | acc     0.45 | train_ae_norm     1.00
[1/200][3499/4361] Loss_D: 1.38639891 (Loss_D_real: 0.69322628 Loss_D_fake: 0.69317263) Loss_G: -0.00001093 Loss_Enh_Dec: 0.00000599
| epoch   1 |  3500/ 4361 batches | lr 0.000000 | ms/batch 396.75 | 

[2/200][1399/4361] Loss_D: 1.38631761 (Loss_D_real: 0.69314563 Loss_D_fake: 0.69317198) Loss_G: 0.00000090 Loss_Enh_Dec: -0.00000929
| epoch   2 |  1400/ 4361 batches | lr 0.000000 | ms/batch 397.12 | loss  4.36 | ppl    78.03 | acc     0.44 | train_ae_norm     1.00
[2/200][1499/4361] Loss_D: 1.38631582 (Loss_D_real: 0.69315672 Loss_D_fake: 0.69315904) Loss_G: 0.00000143 Loss_Enh_Dec: -0.00000251
| epoch   2 |  1500/ 4361 batches | lr 0.000000 | ms/batch 397.41 | loss  4.37 | ppl    79.33 | acc     0.49 | train_ae_norm     1.00
[2/200][1599/4361] Loss_D: 1.38631392 (Loss_D_real: 0.69317311 Loss_D_fake: 0.69314086) Loss_G: 0.00000308 Loss_Enh_Dec: -0.00000895
| epoch   2 |  1600/ 4361 batches | lr 0.000000 | ms/batch 396.61 | loss  4.33 | ppl    76.05 | acc     0.55 | train_ae_norm     1.00
[2/200][1699/4361] Loss_D: 1.38628292 (Loss_D_real: 0.69313288 Loss_D_fake: 0.69314998) Loss_G: -0.00000669 Loss_Enh_Dec: -0.00002164
| epoch   2 |  1700/ 4361 batches | lr 0.000000 | ms/batch 396.29

...

[134/200][1799/4361] Loss_D: 0.00479154 (Loss_D_real: 0.00030795 Loss_D_fake: 0.00448359) Loss_G: 0.44760752 Loss_Enh_Dec: -2.28257990
| epoch 134 |  1800/ 4361 batches | lr 0.000000 | ms/batch 401.81 | loss  2.32 | ppl    10.20 | acc     0.70 | train_ae_norm     1.00
[134/200][1899/4361] Loss_D: 0.00525287 (Loss_D_real: 0.00390362 Loss_D_fake: 0.00134925) Loss_G: 0.47333080 Loss_Enh_Dec: -1.96432877
| epoch 134 |  1900/ 4361 batches | lr 0.000000 | ms/batch 401.36 | loss  2.36 | ppl    10.63 | acc     0.69 | train_ae_norm     1.00
[134/200][1999/4361] Loss_D: 0.00491713 (Loss_D_real: 0.00025276 Loss_D_fake: 0.00466437) Loss_G: 0.49947926 Loss_Enh_Dec: -1.97693121
| epoch 134 |  2000/ 4361 batches | lr 0.000000 | ms/batch 401.91 | loss  2.29 | ppl     9.85 | acc     0.74 | train_ae_norm     1.00
[134/200][2099/4361] Loss_D: 0.00160661 (Loss_D_real: 0.00140654 Loss_D_fake: 0.00020007) Loss_G: 0.57575434 Loss_Enh_Dec: -1.96254122
| epoch 134 |  2100/ 4361 batches | lr 0.000000 | ms/batch


  Average training loss generetor: 0.697
  Average training loss discriminator: 0.698
  Training epcoh took: 0:01:26

Running Test...
  Accuracy: 0.492
  Test Loss: 4.735
  Test took: 0:00:00
Train other shit
| epoch 135 |     0/ 4361 batches | lr 0.000000 | ms/batch 862.46 | loss  0.02 | ppl     1.02 | acc     0.74 | train_ae_norm     1.00
[135/200][99/4361] Loss_D: 0.00246258 (Loss_D_real: 0.00206942 Loss_D_fake: 0.00039316) Loss_G: 0.49520737 Loss_Enh_Dec: -1.77977026
| epoch 135 |   100/ 4361 batches | lr 0.000000 | ms/batch 401.56 | loss  2.18 | ppl     8.88 | acc     0.69 | train_ae_norm     1.00
[135/200][199/4361] Loss_D: 0.00325027 (Loss_D_real: 0.00125289 Loss_D_fake: 0.00199738) Loss_G: 0.42977086 Loss_Enh_Dec: -1.85090530
| epoch 135 |   200/ 4361 batches | lr 0.000000 | ms/batch 401.30 | loss  2.22 | ppl     9.20 | acc     0.77 | train_ae_norm     1.00
[135/200][299/4361] Loss_D: 0.00553098 (Loss_D_real: 0.00150873 Loss_D_fake: 0.00402225) Loss_G: 0.70355409 Loss_Enh_Dec:

| epoch 135 |  3000/ 4361 batches | lr 0.000000 | ms/batch 401.68 | loss  2.23 | ppl     9.26 | acc     0.73 | train_ae_norm     1.00
[135/200][3099/4361] Loss_D: 0.00186326 (Loss_D_real: 0.00032626 Loss_D_fake: 0.00153701) Loss_G: 0.45083547 Loss_Enh_Dec: -2.67118120
| epoch 135 |  3100/ 4361 batches | lr 0.000000 | ms/batch 401.55 | loss  2.21 | ppl     9.16 | acc     0.72 | train_ae_norm     1.00
[135/200][3199/4361] Loss_D: 0.00308181 (Loss_D_real: 0.00115974 Loss_D_fake: 0.00192207) Loss_G: 0.47002441 Loss_Enh_Dec: -2.28861547
| epoch 135 |  3200/ 4361 batches | lr 0.000000 | ms/batch 401.99 | loss  2.27 | ppl     9.67 | acc     0.75 | train_ae_norm     1.00
[135/200][3299/4361] Loss_D: 0.00171173 (Loss_D_real: 0.00158885 Loss_D_fake: 0.00012288) Loss_G: 0.84073633 Loss_Enh_Dec: -2.24127316
| epoch 135 |  3300/ 4361 batches | lr 0.000000 | ms/batch 401.23 | loss  2.29 | ppl     9.85 | acc     0.72 | train_ae_norm     1.00
[135/200][3399/4361] Loss_D: 0.01980634 (Loss_D_real: 0.018

[136/200][1099/4361] Loss_D: 0.00328347 (Loss_D_real: 0.00023525 Loss_D_fake: 0.00304822) Loss_G: 0.41478512 Loss_Enh_Dec: -1.43626010
| epoch 136 |  1100/ 4361 batches | lr 0.000000 | ms/batch 401.33 | loss  2.22 | ppl     9.24 | acc     0.70 | train_ae_norm     1.00
[136/200][1199/4361] Loss_D: 0.01666483 (Loss_D_real: 0.01371816 Loss_D_fake: 0.00294667) Loss_G: 0.43029147 Loss_Enh_Dec: -1.38708448
| epoch 136 |  1200/ 4361 batches | lr 0.000000 | ms/batch 402.09 | loss  2.23 | ppl     9.35 | acc     0.76 | train_ae_norm     1.00
[136/200][1299/4361] Loss_D: 0.00196626 (Loss_D_real: 0.00080932 Loss_D_fake: 0.00115695) Loss_G: 0.52029794 Loss_Enh_Dec: -1.62013018
| epoch 136 |  1300/ 4361 batches | lr 0.000000 | ms/batch 401.56 | loss  2.26 | ppl     9.62 | acc     0.72 | train_ae_norm     1.00
[136/200][1399/4361] Loss_D: 0.00046280 (Loss_D_real: 0.00007985 Loss_D_fake: 0.00038295) Loss_G: 0.48628169 Loss_Enh_Dec: -1.73763025
| epoch 136 |  1400/ 4361 batches | lr 0.000000 | ms/batch

| epoch 136 |  4100/ 4361 batches | lr 0.000000 | ms/batch 401.46 | loss  2.27 | ppl     9.64 | acc     0.71 | train_ae_norm     1.00
[136/200][4199/4361] Loss_D: 0.00198124 (Loss_D_real: 0.00029583 Loss_D_fake: 0.00168541) Loss_G: 0.47951147 Loss_Enh_Dec: -2.11502814
| epoch 136 |  4200/ 4361 batches | lr 0.000000 | ms/batch 401.44 | loss  2.31 | ppl    10.11 | acc     0.72 | train_ae_norm     1.00
[136/200][4299/4361] Loss_D: 0.00231893 (Loss_D_real: 0.00017068 Loss_D_fake: 0.00214825) Loss_G: 0.59530884 Loss_Enh_Dec: -2.36191773
| epoch 136 |  4300/ 4361 batches | lr 0.000000 | ms/batch 401.51 | loss  2.28 | ppl     9.79 | acc     0.70 | train_ae_norm     1.00
| end of epoch 136 | time: 1855.04s | test loss  2.40 | test ppl 11.04 | acc 0.764
bleu_self:  [2.22938381e-01 3.88491377e-09 1.04452896e-11 5.57819826e-13
 9.88323447e-14]
bleu_test:  [8.20486111e-01 9.40558459e-02 6.77875076e-07 1.95407268e-09
 6.27957307e-11]
bleu_self: [0.22293838,0.00000000,0.00000000,0.00000000,0.0000000

In [ ]:
plt.plot(accuracy_array)

In [ ]:
print(max(accuracy_array))
#0.4025 previous best accuracy (bert-base-cased, epoch 50, 5e-6)

In [ ]:
print(accuracy_array[-1])

In [ ]:
df_to_save = pd.DataFrame(accuracy_array)

In [ ]:
df_to_save.to_csv('accuracy_array_assigned_yahoo.csv')